# Investigate results of fitting procedure

In [2]:
import sys
sys.path.append("../python/")

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cmocean
import bokeh.palettes
import _corner
import imp
imp.reload(_corner)
from _corner import corner
import pickle

from matplotlib import rc
rc('font', family='times new roman', size=22.)